
 ### Генерация текста с помощью LSTM RNN на примере пацанских цитат/юморесок.



In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Dropout, CuDNNLSTM, Bidirectional
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.models import Sequential, load_model
import keras.utils as ku 
import numpy as np
import random
import sys
import io

### Настройки

In [0]:
filepath = 'h2.txt'
weights_path = 'weights.hdf5'
new_model = True

### Объявляем keras-модель.

In [0]:
max_sequence_len = 40
layers = 4
hidden_layers = 128
use_dropout = True

In [0]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    for i in range(layers-1):
      model.add(Bidirectional(CuDNNLSTM(hidden_layers, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(hidden_layers)))
    if use_dropout:
      model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

### Объявляем вспомогательные функции

In [0]:
def sample(preds, temperature=1.0):
    """
    Вспомогательная функция для выбора индекса элемента из массива предсказаний.
    preds: предсказания модели
    temperature: коэффициент, определяющий насколько креативны предсказывания модели
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    """Вызывается в конце эпохи. Генерирует текст"""
    
    if epoch+1 == 1 or epoch+1 == 15:
        print()
        print('----- Сгенерированный текст после эпохи: %d' % epoch)

        start_index = random.randint(0, len(data) - maxlen - 1)
        for temperature in [0.2, 0.5, 1.0, 1.2]:
            print('----- temperature:', temperature)

            generated = ''
            sentence = data[start_index: start_index + maxlen]
            generated += sentence
            print('----- : "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()
    else:
        print()
        print('----- Не генерируем текст после эпохи: %d' % epoch)

generate_text = LambdaCallback(on_epoch_end=on_epoch_end)

def generate(sentence='', length=400, temperature=1.0):
  """Генерирует текст длинной length, начинающийся с sentence"""
  sentence = sentence.rjust(maxlen)[:maxlen].lower()
  generated = ''
  generated += sentence

  for i in range(length):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

  return generated

### Открытие и обработка датасета

In [0]:
data = open('h2.txt', 'r').read()

print("Всего %d символов" % len(data))
data = data.lower()
chars = sorted(list(set(data)))
print('Всего уникальных символов:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(data) - maxlen, step):
    sentences.append(data[i: i + maxlen])
    next_chars.append(data[i + maxlen])
print('Всего последовательностей:', len(sentences), "\n")

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


### Тренировка модели

In [0]:
model = create_model(x, y, maxlen, len(chars))

In [0]:
# Чекпоинты. Для сохранения весов и настроек сети
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
if not new_model:
  model = load_model(weights_path)#TODO: разобраться почему не работает

else:
  model.fit(x, y,
          batch_size=256,
          epochs=20,
          verbose=1,
          callbacks=[generate_text, checkpoint])

### Играемся

In [0]:
print(generate("парень с девушкой приходят после свидания к нему домой. Он достает ключи, а девушка ему говорит: ", temperature=0.1, length=500))
#on_epoch_end(0,"")